In [1]:
# from pyspark.sql import SparkSession

In [2]:
# spark = SparkSession.builder.appName('SparkByExample.com').getOrCreate()
#
# data = [("Java", "2000"), ("python", "10000"), ("Scala", "3000")]
# columns = ["language","user_count"]
# df=spark.createDataFrame(data).toDF(*columns)

In [3]:
# df.show()

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dask.dataframe as dd
from pyspark.sql import SparkSession
import multiprocessing as mp
import csv
import datatable as dt
import yaml

import gzip
import os
from subprocess import check_call

# Time to read using pandas (1min 21s)

In [2]:
%%time
NYC_data = pd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2017.csv")
NYC_data.head()

CPU times: user 27.8 s, sys: 15.4 s, total: 43.2 s
Wall time: 1min 21s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN


In [6]:
NYC_data.isnull().sum()

Summons Number                              0
Plate ID                                  728
Registration State                          0
Plate Type                                  0
Issue Date                                  0
Violation Code                              0
Vehicle Body Type                       42711
Vehicle Make                            73050
Issuing Agency                              0
Street Code1                                0
Street Code2                                0
Street Code3                                0
Vehicle Expiration Date                     0
Violation Location                    2072400
Violation Precinct                          0
Issuer Precinct                             0
Issuer Code                                 0
Issuer Command                        2062645
Issuer Squad                          2063541
Violation Time                             63
Time First Observed                   9962281
Violation County                  

In [7]:
NYC_data.shape

(10803028, 43)

# Removing White spaces and special characters from columns names
Time to read using Modin is much lesser than pandas (Not able to show modin results due to some error)

In [8]:
NYC_data.columns = NYC_data.columns.str.replace('[?,@,#]','')

In [9]:
NYC_data.columns = NYC_data.columns.str.replace('[ ]','_')

In [10]:
NYC_data.head()

,Summons_Number,Plate_ID,Registration_State,Plate_Type,Issue_Date,Violation_Code,Vehicle_Body_Type,Vehicle_Make,Issuing_Agency,Street_Code1,...,Vehicle_Color,Unregistered_Vehicle,Vehicle_Year,Meter_Number,Feet_From_Curb,Violation_Post_Code,Violation_Description,No_Standing_or_Stopping_Violation,Hydrant_Violation,Double_Parking_Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN


# Time to read using pandas chunks (46s)

In [11]:
%%time
data_chunks = pd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2017.csv", chunksize=100000)
pandas_chunks = pd.concat(data_chunks)
pandas_chunks.head()

CPU times: user 29.9 s, sys: 8.71 s, total: 38.6 s
Wall time: 46.5 s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN


# Time to read using DASK(2s)

In [6]:
%%time 
data_dask = dd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2017.csv", dtype={'House Number': 'object',
       'Time First Observed': 'object'})
data_dask.head()

CPU times: user 882 ms, sys: 382 ms, total: 1.26 s
Wall time: 2.07 s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN


# Time to read using datatable(6.5s)

In [14]:
%%time 
data_dt = dt.fread("Parking_Violations_Issued_-_Fiscal_Year_2017.csv")
data_dt.head()

CPU times: user 16.3 s, sys: 2.71 s, total: 19 s
Wall time: 6.5 s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,…,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪,▪,▪
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,…,,FAILURE TO STOP AT RED LIGHT,NA,NA,NA
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,…,,FAILURE TO STOP AT RED LIGHT,NA,NA,NA
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,…,,BUS LANE VIOLATION,NA,NA,NA
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,…,04,47-Double PKG-Midtown,NA,NA,NA
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,…,31 6,69-Failure to Disp Muni Recpt,NA,NA,NA
5,5096917368,FZD8593,NY,PAS,06/13/2017,7,SUBN,ME/BE,V,0,…,,FAILURE TO STOP AT RED LIGHT,NA,NA,NA
6,1413609545,X20DCM,NJ,PAS,08/03/2016,40,SDN,TOYOT,P,54070,…,,,NA,NA,NA
7,4628525523,326SF9,MA,PAS,12/21/2016,36,UT,BMW,V,0,…,,PHTO SCHOOL ZN SPEED VIOLATION,NA,NA,NA
8,4627113330,HCA5464,NY,OMS,11/21/2016,36,SUBN,DODGE,V,0,…,,PHTO SCHOOL ZN SPEED VIOLATION,NA,NA,NA


# Time to read using pyspark (32.8s)

In [15]:
%%time
spark = SparkSession.builder.appName("EDA.com").getOrCreate()
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Parking_Violations_Issued_-_Fiscal_Year_2017.csv")
df.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/09 12:08:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/09 12:08:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Is

# Time to read using dictreader(3.45ms)

In [16]:
%%time
data_dict = csv.DictReader(open('Parking_Violations_Issued_-_Fiscal_Year_2017.csv'))
i=0
for row in data_dict:
    print(row)
    i+=1
    if i is 5:
        break

{'Summons Number': '5092469481', 'Plate ID': 'GZH7067', 'Registration State': 'NY', 'Plate Type': 'PAS', 'Issue Date': '07/10/2016', 'Violation Code': '7', 'Vehicle Body Type': 'SUBN', 'Vehicle Make': 'TOYOT', 'Issuing Agency': 'V', 'Street Code1': '0', 'Street Code2': '0', 'Street Code3': '0', 'Vehicle Expiration Date': '0', 'Violation Location': '', 'Violation Precinct': '0', 'Issuer Precinct': '0', 'Issuer Code': '0', 'Issuer Command': '', 'Issuer Squad': '', 'Violation Time': '0143A', 'Time First Observed': '', 'Violation County': 'BX', 'Violation In Front Of Or Opposite': '', 'House Number': '', 'Street Name': 'ALLERTON AVE (W/B) @', 'Intersecting Street': 'BARNES AVE', 'Date First Observed': '0', 'Law Section': '1111', 'Sub Division': 'D', 'Violation Legal Code': 'T', 'Days Parking In Effect    ': '', 'From Hours In Effect': '', 'To Hours In Effect': '', 'Vehicle Color': 'GY', 'Unregistered Vehicle?': '', 'Vehicle Year': '2001', 'Meter Number': '', 'Feet From Curb': '0', 'Violati